# Installs and imports

In [ ]:
%%capture
!pip install netCDF4;
!pip install visualkeras

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import netCDF4
import pandas as pd
from datetime import datetime
import seaborn as sns
from sklearn.metrics import r2_score,mean_squared_error
from google.colab import files
from keras.callbacks import Callback
import time
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from keras.wrappers.scikit_learn import KerasRegressor
import tensorflow as tf
from keras.layers import Conv2D, MaxPooling2D, concatenate, Flatten, Dense, Dropout, Input
from keras.models import Model
import os
import math

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# Read CSV

In [ ]:
df_robot = pd.read_csv('/info_file/robot_info.csv', sep=';')
df_nobal = pd.read_csv('/info_file/nobal_info.csv', sep=';')

In [ ]:
# Extract numeric part from a column and convert it to numeric type
df_nobal['N_level'] = df_nobal['N_level'].str.extract(r'(\d+)').astype(float)
df_robot['N_level'] = df_robot['N_level'].str.extract(r'(\d+)').astype(float)

In [ ]:
dates_list = ['16.05.2022', '28.06.2022', '21.07.2022', '01.08.2022', '01.09.2022', '20.05.2022', '28.07.2022', '05.08.2022', '31.05.2022', '05.08.2022', '30.08.2022']

In [ ]:
mean_value_dth = df_nobal['dth'].mean()
df_nobal['dth'].fillna(mean_value_dth, inplace=True)
df_nobal['dth'].isna().sum()

0

In [ ]:
# Flatten the list of dates
dates_list = ['16.05.2022', '28.06.2022', '21.07.2022', '01.08.2022', '01.09.2022', '20.05.2022', '28.07.2022', '05.08.2022', '31.05.2022', '05.08.2022', '30.08.2022']

# Convert the list of dates to datetime type
flat_dates_list = [datetime.strptime(date, '%d.%m.%Y') for date in dates_list]
flat_dates_list.sort()

# Convert the 'dates' column to datetime type
df_robot['HD_day'] = pd.to_datetime(df_robot['HD_day'], format='%d.%m.%Y')
days_between = []


# Iterate over each date in the flat_dates_list and calculate the days between the date and all dates in the DataFrame
for date in flat_dates_list:
    days = (df_robot['HD_day'] - date).dt.days
    days_between.append(days)

# Convert the 'dates' column to datetime type
df_nobal['HD_day'] = pd.to_datetime(df_nobal['HD_day'], format='%d.%m.%Y')


In [ ]:
mean_value_hd = df_nobal['HD_day'].mean()
df_nobal['HD_day'].fillna(mean_value_hd, inplace=True)
df_nobal['HD_day'].isna().sum()
days_between_nobal = []

# Iterate over each date in the flat_dates_list and calculate the days between the date and all dates in the DataFrame
for date in flat_dates_list:
    days = (df_nobal['HD_day'] - date).dt.days
    days_between_nobal.append(days)

for item in days_between_nobal:
  for number in item:
    if math.isnan(number):
        print("NaN found")

In [ ]:

# Concatenate 'N_level' and 'dth' arrays from 'df_nobal' and 'df_robot'
nobal_n_level = np.concatenate([df_nobal['N_level'].values, df_nobal['N_level'].values, df_nobal['N_level'].values])
nobal_dth = np.concatenate([df_nobal['dth'].values, df_nobal['dth'].values, df_nobal['dth'].values])
robot_n_level = np.concatenate([df_robot['N_level'].values, df_robot['N_level'].values, df_robot['N_level'].values])
robot_dth = np.concatenate([df_robot['dth'].values, df_robot['dth'].values, df_robot['dth'].values])

# Get the length of the resulting arrays
len_nobal_n_level = len(nobal_n_level)
len_nobal_dth = len(nobal_dth)
len_robot_n_level = len(robot_n_level)
len_robot_dth = len(robot_dth)

# Print the lengths
print("Length of nobal_n_level:", len_nobal_n_level)
print("Length of nobal_dth:", len_nobal_dth)
print("Length of robot_n_level:", len_robot_n_level)
print("Length of robot_dth:", len_robot_dth)

Length of nobal_n_level: 192
Length of nobal_dth: 192
Length of robot_n_level: 288
Length of robot_dth: 288


# CNN

In [ ]:
from tensorflow.keras.utils import plot_model
from tensorflow import keras
def create_model(num_filters=128, kernel_size=3, pool_size=2, dropout_rate=0.2):
    # Define input shapes
    images_input_shape = (256, 52, 5)  # Input shape for 'images'
    days_between_input_shape = (1)  # Input shape for 'dth'

    # Create input layers
    images_input = Input(shape=images_input_shape, name='images_input')
    days_between_input = Input(shape=days_between_input_shape, name='days_between_input')
  
   
    # Add convolutional and pooling layers
    conv1 = Conv2D(num_filters, kernel_size=(kernel_size, kernel_size), activation='relu', padding='same')(images_input)
    conv2 = Conv2D(num_filters, kernel_size=(kernel_size, kernel_size), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(num_filters, kernel_size=(kernel_size, kernel_size), activation='relu', padding='same')(pool1)
    conv4 = Conv2D(num_filters, kernel_size=(kernel_size, kernel_size), activation='relu', padding='same')(conv3)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv4)
    conv5 = Conv2D(num_filters, kernel_size=(kernel_size, kernel_size), activation='relu', padding='same')(pool2)
    conv6 = Conv2D(num_filters, kernel_size=(kernel_size, kernel_size), activation='relu', padding='same')(conv5)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv6)

    # Flatten the feature maps
    flatten = Flatten()(pool3)

    # Concatenate the input layers along the channel axis
    concatenated = concatenate([flatten, days_between_input], axis=-1)

    # Add fully connected layers
    dense1 = Dense(512, activation='relu')(concatenated)
    dropout1 = Dropout(dropout_rate)(dense1)

    # Define the model with grain yield as output
    gy_output = Dense(units=1, name='gy_output')(dropout1)

    # Create the model
    model = Model(inputs=[images_input, days_between_input], outputs=[gy_output])
    opt = keras.optimizers.Adam(learning_rate=0.00001)
    # Compile the model
    model.compile(optimizer=opt, loss='mse', metrics=['mse'])
    #visualkeras.layered_view(model, legend=True).show()
    plot_model(model, to_file='cnn_concat.pdf', show_shapes=True)

    return model

In [ ]:
from keras.layers import Conv2D, MaxPooling2D, concatenate, Flatten, Dense, Dropout, Input
from keras.models import Model

# Define a function that returns the CNN model
def create_model_n_level(num_filters=128, kernel_size=3, pool_size=2, dropout_rate=0.2, padding='same'):
    # Define input shapes
    images_input_shape = (256, 52, 5)  # Input shape for 'images'
    fertilization_input_shape = (1)  # Input shape for 'fertilization'
    days_between_input_shape = (1)  # Input shape for 'dth'

    # Create input layers
    images_input = Input(shape=images_input_shape, name='images_input')
    fertilization_input = Input(shape=fertilization_input_shape, name='fertilization_input')
    days_between_input = Input(shape=days_between_input_shape, name='days_between_input')
  
   
    # Add convolutional and pooling layers
    conv1 = Conv2D(num_filters, kernel_size=(kernel_size, kernel_size), activation='relu', padding=padding)(images_input)
    conv2 = Conv2D(num_filters, kernel_size=(kernel_size, kernel_size), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(num_filters, kernel_size=(kernel_size, kernel_size), activation='relu', padding='same')(pool1)
    conv4 = Conv2D(num_filters, kernel_size=(kernel_size, kernel_size), activation='relu', padding='same')(conv3)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv4)
    conv5 = Conv2D(num_filters, kernel_size=(kernel_size, kernel_size), activation='relu', padding='same')(pool2)
    conv6 = Conv2D(num_filters, kernel_size=(kernel_size, kernel_size), activation='relu', padding='same')(conv5)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv6)

    # Flatten the feature maps
    flatten = Flatten()(pool3)

    # Concatenate the input layers along the channel axis
    concatenated = concatenate([flatten, fertilization_input, days_between_input], axis=-1)

    # Add fully connected layers
    dense1 = Dense(512, activation='relu')(concatenated)
    dropout1 = Dropout(dropout_rate)(dense1)

    # Define the model with grain yield as output
    gy_output = Dense(units=1, name='gy_output')(dropout1)

    # Create the model
    model = Model(inputs=[images_input, fertilization_input, days_between_input], outputs=[gy_output])

    # Compile the model
    opt = keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(optimizer=opt, loss='mse', metrics=['mse'])
    #plot_model(model, to_file='cnn_concat_n_level.pdf', show_shapes=True)
    return model

## Hyperparameters

In [ ]:
epochs = 1
num_filters=128
kernel_size=2
pool_size=3
dropout_rate=0.2
batch_size = 16

# Function for visualization

In [ ]:
def residualplot(y_pred, y_true, dsh, figurename):
    # Set fontsize
    xlim_max = 8
    xlim_min = 3
    ylim_max = 5
    ylim_min = -5

    # Calculate residuals and their z-scores
    residuals = y_true - y_pred
    residual_mean = np.mean(residuals)
    residual_std = np.std(residuals)
    z_scores = (residuals - residual_mean) / residual_std

    # Create residual plots using Seaborn
    plt.figure(figsize=(10, 8))
    sns.residplot(x=y_pred, y=z_scores, scatter_kws={'alpha': 0.5}, hue=dsh)
    plt.xlabel('Predicted Values', fontsize=18)
    plt.ylabel('Standardized Residuals', fontsize=18)
    plt.title(f'Residual Plot (Model {figurename})',fontsize=18)
    plt.xticks(fontsize=18)
    plt.yticks(fontsize=18)
    plt.ylim(ylim_min, ylim_max)  # Set y-axis range
    plt.xlim(xlim_min, xlim_max)  # Set x-axis range
    plt.tight_layout()  # Add
    plt.savefig(f'resplot_Model_{figurename}.pdf', bbox_inches='tight')
    files.download(f'resplot_Model_{figurename}.pdf')
    plt.show()

In [ ]:
def scatterplot(y_pred, y_true, r2, mse, mbd, dsh, figurename):
    # Create scatter plot with Seaborn for nobal
    plt.figure(figsize=(10, 8))
    plt.rcParams['legend.fontsize'] = 18
    sns.scatterplot(x=y_true, y=y_pred, color='#8bad84', s=70, hue=dsh)
    plt.xlabel('Measured grain yield (t/ha)', fontsize=18)
    plt.ylabel('Predicted grain yield (t/ha)', fontsize=18)
    plt.title(f'Actual vs. Predicted Values (Model {figurename})', fontsize=18)
    sns.lineplot(x=[4.2, 7.8], y=[4.2, 7.8])
    plt.xticks(fontsize=18)
    plt.yticks(fontsize=18)
    plt.ylim(4, 8)  # Set y-axis range
    plt.xlim(4, 8)  # Set x-axis range
    plt.annotate('MSE: {:.2f}'.format(mse), (4.2, 7.8), ha='left', va='top', fontsize=16)  # Add MBD as text annotation
    plt.annotate(f'R-squared: {r2:.3f}', (4.2, 7.6), ha='left', va='top', fontsize=16)  # Add R-squared as text annotation
    #plt.annotate(f'MBE: {mbd:.3f}', (4.2, 7.4), ha='left', va='top', fontsize=16)  # Add R-squared as text annotation
    plt.legend(loc='upper left', bbox_to_anchor=(1.02, 1), borderaxespad=0, fontsize=16)  # Add legend outside the plot
    plt.savefig(f'scatter_Model_{figurename}.pdf', bbox_inches='tight')
    files.download(f'scatter_Model_{figurename}.pdf')
    plt.show()

In [ ]:
def normalize(X):
  X_min = np.min(X)
  X_max = np.max(X)
  X_norm = (X - X_min) / (X_max - X_min)
  return X_norm

# Read all

In [ ]:
# Initialize empty lists to store X and y
X_all_dth_date = []
y_all_dth_date = []
dates = []

# Loop through the files
for i in range(1,8):
    # Read data from h5 file
    with h5py.File(f'/datasets/datetime_robot_{i}.h5', 'r') as hf:
        # Append data to X_list and y_list
        X_all_dth_date.extend(hf['X'][:])
        y_all_dth_date.extend(hf['y'][:])

    # Extend date_captured list with three days between
    dates.extend([days_between[i], days_between[i], days_between[i]])

# Convert X_list_dth_date and y_dth_date to numpy arrays
X = np.array(X_all_dth_date)
y_all_dth_date = np.array(y_all_dth_date)
X = np.transpose(X_all_dth_date, (0,2,3,1))
dates = np.array(dates).flatten()
# Apply min-max normalization to the X data

X_norm_all = normalize(X)

In [ ]:
X_train, X_test, y_train, y_test_all, days_train, days_test = train_test_split(X_norm_all, y_all_dth_date, dates, test_size=0.2, random_state=42, shuffle=True)

In [ ]:
early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=50)
model = create_model(num_filters=num_filters, kernel_size=kernel_size, pool_size=pool_size, dropout_rate=dropout_rate)
model_history = model.fit([X_train, days_train], y_train,
                    batch_size=batch_size, 
                    epochs=epochs, 
                    validation_split=0.2,
                    callbacks=[early_stopping_callback], 
                    shuffle=True)

81/81 [==============================] - 17s 72ms/step - loss: 16.5894 - mse: 16.5894 - val_loss: 2.6458 - val_mse: 2.6458


In [ ]:
pred_all = model.predict([X_test, days_test], batch_size=batch_size)


26/26 [==============================] - 1s 19ms/step


# Read one

In [ ]:
mse_one_scores = []
datetime = []
date = 8

# Create an instance of the custom callback
with h5py.File(f'/datasets/datetime_robot_{date}.h5', 'r') as hf:
    print(hf.keys())
    X = hf['X'][:]
    y = hf['y'][:]



datetime.extend([days_between[date], days_between[date], days_between[date]])
datetime = np.array(datetime).flatten()
X = np.array(X)               
X = np.transpose(X, (0,2,3,1))
y = np.array(y)


X_norm_one = normalize(X)

In [ ]:
early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=100)
X_train, X_test, y_train, y_test_one, datetime_train, datetime_test = train_test_split(X_norm_one, y, datetime, test_size=0.2, random_state=42, shuffle=True)

In [ ]:
# Split the combined data into training and testing sets
model = create_model(num_filters=num_filters, kernel_size=kernel_size, pool_size=pool_size, dropout_rate=dropout_rate)
model_history = model.fit([X_train, datetime_train], y_train,
                  batch_size=batch_size, 
                  epochs=epochs, 
                  validation_split=0.2,
                  callbacks=[early_stopping_callback], 
                  shuffle=True)

In [ ]:
pred_one = model.predict([X_test, datetime_test], batch_size=batch_size)

In [ ]:
# Create a dataframe to store the predicted and true values
df = pd.DataFrame({'predicted': pred_one.reshape(-1), 'true': y_test_one.reshape(-1),  'DSH': datetime_test})

# Write the dataframe to a CSV file
df.to_csv('/csv_files/predicted_vs_true_SingleDateTimeRobot.csv', index=False)

# Nobal in training, Robot in test

In [ ]:
# Initialize empty lists to store X and y
X_list_robot = []
y_list_robot = []
date_captured_robot = []

X_list_nobal = []
y_list_nobal = []
date_captured_nobal = []

# Loop through the files
for i in range(1,9):
    # Read data from h5 file
    with h5py.File(f'/datasets/datetime_robot_{i}.h5', 'r') as robot:
        # Append data to X_list and y_list
        X_list_robot.extend(robot['X'][:])
        y_list_robot.extend(robot['y'][:])
    with h5py.File(f'/datasets/datetime_nobal_{i}.h5', 'r') as nobal:
      X_list_nobal.extend(nobal['X'][:])
      y_list_nobal.extend(nobal['y'][:])

    date_captured_robot.extend([days_between[i], days_between[i], days_between[i]])
    date_captured_nobal.extend([days_between_nobal[i], days_between_nobal[i], days_between_nobal[i]])

# Convert X_list_dth_date and y_dth_date to numpy arrays
X_list_robot = np.array(X_list_robot)
X_list_robot = np.transpose(X_list_robot, (0,2,3,1))
y_list_robot = np.array(y_list_robot)
date_captured_robot = np.array(date_captured_robot).flatten()

X_list_nobal = np.array(X_list_nobal)
X_list_nobal = np.transpose(X_list_nobal, (0,2,3,1))
y_list_nobal = np.array(y_list_nobal).squeeze()
date_captured_nobal = np.array(date_captured_nobal).flatten()


X_norm_robot = normalize(X_list_robot)
X_norm_nobal = normalize(X_list_nobal)

In [ ]:
early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10)
model = create_model(num_filters=num_filters, kernel_size=kernel_size, pool_size=pool_size, dropout_rate=dropout_rate)
model_history = model.fit([X_norm_nobal, date_captured_nobal], y_list_nobal,
                  batch_size=batch_size, 
                  epochs=epochs, 
                  verbose=2,
                  callbacks=[early_stopping_callback], 
                  validation_split=0.2, 
                  shuffle=True)

In [ ]:
pred_only_nobal = model.predict([X_norm_robot ,date_captured_robot], batch_size=batch_size)

In [ ]:
# Create a dataframe to store the predicted and true values
df = pd.DataFrame({'predicted': pred_only_nobal.reshape(-1), 'true': y_list_robot.reshape(-1), 'DSH': date_captured_robot})

# Write the dataframe to a CSV file
df.to_csv('/csv_files/predicted_vs_true_SeparatedDataTestRobot.csv', index=False)

# Mixed Data

In [ ]:
# Initialize empty lists to store X and y
X_list_robot = []
y_list_robot = []
date_captured_robot = []
robot_n_level_date = []

X_list_nobal = []
y_list_nobal = []
date_captured_nobal = []
nobal_n_level_date = []

# Loop through the files
for i in range(1,9):
    # Read data from h5 file
    with h5py.File(f'/datasets/datetime_robot_{i}.h5', 'r') as robot:
        # Append data to X_list and y_list
        X_list_robot.extend(robot['X'][:])
        y_list_robot.extend(robot['y'][:])
        robot_n_level_date.extend(robot_n_level)
    with h5py.File(f'/datasets/nobal_datetime_{i}.h5', 'r') as nobal:
      X_list_nobal.extend(nobal['X'][:])
      y_list_nobal.extend(nobal['y'][:])

    date_captured_robot.extend([days_between[i], days_between[i], days_between[i]])
    date_captured_nobal.extend([days_between_nobal[i], days_between_nobal[i], days_between_nobal[i]])
    
    

# Convert X_list_dth_date and y_dth_date to numpy arrays
X_list_robot = np.array(X_list_robot)
X_list_robot = np.transpose(X_list_robot, (0,2,3,1))
y_list_robot = np.array(y_list_robot)
date_captured_robot = np.array(date_captured_robot).flatten()

X_list_nobal = np.array(X_list_nobal)
X_list_nobal = np.transpose(X_list_nobal, (0,2,3,1))
y_list_nobal = np.array(y_list_nobal).squeeze()
date_captured_nobal = np.array(date_captured_nobal).flatten()

X_both = np.concatenate((X_list_nobal, X_list_robot))
y_both = np.concatenate((y_list_nobal, y_list_robot))
date_captured_both = np.concatenate((date_captured_nobal, date_captured_robot))


X_norm_both = normalize(X_both)

In [ ]:
early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=100)
X_train, X_test_both, y_train, y_test_nobal, date_captured_both_train, date_captured_both_test = train_test_split(X_norm_both, y_both, date_captured_both, test_size=0.2, random_state=42, shuffle=True)

model = create_model(num_filters=num_filters, kernel_size=kernel_size, pool_size=(2, 2, 1), dropout_rate=dropout_rate)
model_history = model.fit([X_train, date_captured_both_train], y_train,
                batch_size=batch_size, 
                epochs=epochs, 
                callbacks=[early_stopping_callback], 
                validation_split=0.2, 
                shuffle=True)

pred_nobal = model.predict([X_test_both, date_captured_both_test], batch_size=batch_size)

In [ ]:
# Create a dataframe to store the predicted and true values
df = pd.DataFrame({'predicted': pred_nobal.reshape(-1), 'true': y_test_nobal.reshape(-1), 'DSH': date_captured_both_test})

# Write the dataframe to a CSV file
df.to_csv('csv_files/predicted_vs_true_MixedData.csv', index=False)

# DTH and Fertilization

In [ ]:
# Initialize empty lists to store X and y
X_list_dth_date = []
y_dth_date = []
date_captured = []
#robot_dth_date = []
robot_n_level_date = []
# Loop through the files
for i in range(1,9):
    # Read data from h5 file
    with h5py.File(f'/datasets/datetime_robot_{i}.h5', 'r') as hf:
        # Append data to X_list_dth_date and y_dth_date
        X_list_dth_date.extend(hf['X'][:])
        y_dth_date.extend(hf['y'][:])
        
    # Extend date_captured list with three days between
    date_captured.extend([days_between[i], days_between[i], days_between[i]])
    #robot_dth_date.extend(robot_dth)
    robot_n_level_date.extend(robot_n_level)


# Convert X_list_dth_date and y_dth_date to numpy arrays
X_dth_date = np.array(X_list_dth_date)
y_dth_date = np.array(y_dth_date)
X_dth_date = np.transpose(X_dth_date, (0,2,3,1))

X_norm_dth = normalize(X_dth_date)

data_captured_dth = np.array(date_captured).flatten().reshape(-1, 1)
#robot_dth_date = np.array(robot_dth_date).reshape(-1, 1)
robot_n_level_date = np.asarray(robot_n_level_date).reshape(-1, 1)

In [ ]:
# Train test split
X_dth_date_train, X_dth_date_test, y_dth_date_train, y_dth_date_test, data_captured_dth_train, data_captured_dth_test, robot_n_level_date_train, robot_n_level_date_test = train_test_split(X_norm_dth, y_dth_date, data_captured_dth, 
                                           robot_n_level_date,  test_size=0.2, random_state=42, shuffle=True)


In [ ]:
early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=50)

In [ ]:
model = create_model_n_level(num_filters=num_filters, kernel_size=kernel_size, pool_size=2, dropout_rate=dropout_rate)
model_history = model.fit([X_dth_date_train, robot_n_level_date_train, data_captured_dth_train], y_dth_date_train, 
                        batch_size=batch_size, 
                        epochs=epochs, 
                        callbacks=[early_stopping_callback], 
                        validation_split=0.2, 
                        shuffle=True)

93/93 [==============================] - 9s 70ms/step - loss: 3.9174 - mse: 3.9174 - val_loss: 0.8298 - val_mse: 0.8298


In [ ]:
pred_dth_datetime = model.predict([X_dth_date_test, robot_n_level_date_test, data_captured_dth_test], batch_size=batch_size)

29/29 [==============================] - 1s 26ms/step


In [ ]:
# Create a dataframe to store the predicted and true values
df = pd.DataFrame({'predicted': pred_dth_datetime.reshape(-1), 'true': y_dth_date_test.reshape(-1), 'DSH': data_captured_dth_test.squeeze()})

# Write the dataframe to a CSV file
df.to_csv('/csv_files/predicted_vs_true_DateTimeConcatenationRobot_new.csv', index=False)

# Visualization

In [ ]:
# assign directory
directory = '/csv_files'

# iterate over files in that directory
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        # extract the last name of the file without extension
        last_name = os.path.splitext(os.path.basename(filename))[0]
        # split the last name by '_'
        name_parts = last_name.split('_')
        # get the last part of the name
        plot_name = name_parts[-2]
        df = pd.read_csv(f)
        y_true = df['true']
        y_predicted = df['predicted']
        dsh = df['DSH']

        residuals_one = y_true - y_predicted
        r2 = r2_score(y_true, y_predicted)
        mse = mean_squared_error(y_true, y_predicted)
        mbd = np.mean(y_predicted - y_true)

        scatterplot(y_predicted, y_true, r2, mse, mbd, dsh,plot_name)
        residualplot(y_predicted, y_true, plot_name)